# Start of mess around

In [1]:
from __future__ import print_function

import numpy as np 

import matplotlib.pyplot as plt   # use matplotlib for plotting with inline plots
plt.set_cmap('jet');
%matplotlib inline

import warnings
warnings.filterwarnings('ignore'); # for deprecated matplotlib functions

import math

import tensorflow as tf
from tensorflow.keras import Model, layers


# Interesting imports
In order to import mltools properly, we need to modify the module path that python thinks we have

Basically it adds everything 2 folders up onto our python module path, so referencing "mltools" python can see that it is two directories up

In [2]:
# We need to append the module path so we can work with
#     everything that is 2 folders up
import sys
import os
from pathlib import Path

path = Path(os.getcwd())
project_dir = str(path.parent.parent)

curr_path = sys.path
data_path = ""


In [3]:
if project_dir not in curr_path:
    print("Appending'", project_dir, "'to path")
    sys.path.append(project_dir)
    print("Appending'", project_dir + os.path.sep + "data '", "to path")
    sys.path.append(project_dir + os.path.sep + "data")
    data_path = project_dir + os.path.sep + "data" + os.path.sep
    
# Uncomment this if youd like to see all your python module paths
# print(sys.path)

import mltools as ml

Appending' /home/chris/git/ML-FinalProject 'to path
Appending' /home/chris/git/ML-FinalProject/data ' to path


# Import the data

In [8]:
# print(sys.path)
# print(data_path)
X_train = np.genfromtxt(data_path + "X_train.txt", delimiter=None, skip_header=1)
X_test = np.genfromtxt(data_path + "X_test.txt", delimiter=None, skip_header=1)
Y_train = np.genfromtxt(data_path + "Y_train.txt", delimiter=None, skip_header=1)
Data_Labels = np.genfromtxt(data_path + "X_train.txt", delimiter=None, dtype=str, max_rows=1)

print("Data Labels with Column index:\n")
for i in range(len(Data_Labels)):
    print(str(i) + ": " + str(Data_Labels[i]))


Data Labels with Column index:

0: UniqueID
1: SIZE
2: BIG4
3: GAAP
4: CR
5: QR
6: AT
7: CashR
8: NWC
9: CFOCL
10: OM
11: TM
12: CFM
13: ROE
14: ROA
15: GRIE
16: CFONI
17: CFOTL
18: CFOTR
19: CFOTA
20: EF
21: FAF
22: LTDE
23: CAT
24: DCOH
25: ACP
26: AAP
27: DR
28: DSCR
29: CFOLTL
30: TAT
31: FAT
32: DEPR
33: CFOFA
34: APP
35: 2008
36: 2009
37: 2010
38: 2011
39: 2012
40: 2013
41: 2014
42: 2015
43: 2016
44: 2017
45: 2018


## Lets try TensorFlow

In [9]:
# Network Parameters
n_hidden_1 = 128 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons

# Split the data
cols_using = [1, 2, 3, 5, 8]
Xtrain,Xval,Ytrain,Yval = ml.splitData(X_train[:, cols_using],Y_train[:, 0], 0.75) # split data set 75/25

Xtrain, Xval = np.array(Xtrain, np.float32), np.array(Xval, np.float32)

# Data Parameters
num_features = 47
num_classes = 2 # We have 2 classes, yes or no

# Parameters
learning_rate = 0.1    # alpha for gradient descent
num_steps = 2000       # iterations for gradient descent
batch_size = 256       # number of inputs to look at simultaneously (good for large data!)
display_step = 100     # when to print out some feedback

print("Columns we are currently using:", Data_Labels[cols_using])

print(Ytrain.shape)
print(Xtrain.shape)

# Use tf.data API to shuffle and batch data (batches will make it faster to queue up 
# sets of images for training all at one time--a convenient way to split up large data sets)
train_data = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)


Columns we are currently using: ['SIZE' 'BIG4' 'GAAP' 'QR' 'NWC']
(3582,)
(3582, 5)


## Create our NN model, this inherits from TF

In [10]:
# Create TF Model. Our NeuralNet inherits from the generic TF Model class
class NeuralNet(Model):
    # Set layers.
    def __init__(self):
        super(NeuralNet, self).__init__()
        curr_activation = tf.nn.relu
        # First fully-connected hidden layer. Activation function (threshold function) is 
        # rectified linear.
        self.h1 = layers.Dense(n_hidden_1, activation=curr_activation)
        
        # First fully-connected hidden layer. Activation function (threshold function) is 
        # rectified linear.
        self.h2 = layers.Dense(n_hidden_2, activation=curr_activation)
        
        # Second fully-connecter hidden layer. Activation function is using "softmax" to
        # normalize output as a probability distribution over the 2 classes (digits 0 or 1)
        self.out = layers.Dense(num_classes, activation=tf.nn.softmax)

    # Set forward pass--this defines the input layer (h1) and output layer (out) and then
    # formats the final results as a probability distribution over the classes using softmax
    def call(self, x, is_training=False):
        x = self.h1(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits (positions on the logistic regression sigmoid)
            # without softmax normalization, so only apply softmax when not training.
            x = tf.nn.softmax(x)
        return x
    
# Build neural network model.
neural_net = NeuralNet()


## Specify Loss

In [11]:
# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits as part of the function, so don't do
# it before calling.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric. This counts how many of our predictions we get right based on 
# choosing the prediction with the highest probability. 
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

## Attempt to train?

In [13]:
tf.keras.backend.clear_session()

# Build neural network model.
neural_net = NeuralNet()
# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(.001)

# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation 
    # (see Backprop.ipnb for an explanation of the GradientTape)
    with tf.GradientTape() as g:
        # Forward pass.
        pred = neural_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = neural_net.trainable_variables

    # Compute gradients (backpropagation).
    gradients = g.gradient(loss, trainable_variables)
    
    # Update all of the weights W and biases (y-intercepts) b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    
num_steps = 6000
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(num_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0 or step==1:
        pred = neural_net(batch_x, is_training=True)
#         print("Shape of pred:", pred.shape)
#         print("Shape of batch:", batch_y.shape)
#         print(pred[1, 0])
#         print(pred[1, 1])
#         print()
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 1, loss: nan, accuracy: 0.042969
step: 100, loss: nan, accuracy: 0.023438
step: 200, loss: nan, accuracy: 0.027344
step: 300, loss: nan, accuracy: 0.031250
step: 400, loss: nan, accuracy: 0.031250
step: 500, loss: nan, accuracy: 0.019531
step: 600, loss: nan, accuracy: 0.027344
step: 700, loss: nan, accuracy: 0.019531
step: 800, loss: nan, accuracy: 0.039062
step: 900, loss: nan, accuracy: 0.042969
step: 1000, loss: nan, accuracy: 0.035156
step: 1100, loss: nan, accuracy: 0.035156
step: 1200, loss: nan, accuracy: 0.031250
step: 1300, loss: nan, accuracy: 0.027344
step: 1400, loss: nan, accuracy: 0.035156
step: 1500, loss: nan, accuracy: 0.031250
step: 1600, loss: nan, accuracy: 0.035156
step: 1700, loss: nan, accuracy: 0.050781
step: 1800, loss: nan, accuracy: 0.023438
step: 1900, loss: nan, accuracy: 0.023438
step: 2000, loss: nan, accuracy: 0.066406
step: 2100, loss: nan, accuracy: 0.031250
step: 2200, loss: nan, accuracy: 0.015625
step: 2300, loss: nan, accuracy: 0.035156
step

## Test

In [14]:
# Test model on validation set.
pred = neural_net(Xval, is_training=False)
print("Test Accuracy: %f" % accuracy(pred, Yval))

Test Accuracy: 0.034338
